In [2]:
import json
import pandas as pd
import requests

popular = []
for i in range(1,1000):
    url = 'https://api.themoviedb.org/3/movie/popular?api_key={1}&language=ko-KR&page={0}'.format(i,'36927ad1d2817ff0fa31947e47d186fd')
    resp = requests.get(url)
    popular.append(json.loads(resp.content))

    if i == 10:
        break

popular_movies = []
for row in popular:
    for result in row ['results']:
        popular_movies.append(result)

In [3]:
popular_movies=pd.DataFrame(popular_movies)
popular_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              200 non-null    bool   
 1   backdrop_path      199 non-null    object 
 2   genre_ids          200 non-null    object 
 3   id                 200 non-null    int64  
 4   original_language  200 non-null    object 
 5   original_title     200 non-null    object 
 6   overview           200 non-null    object 
 7   popularity         200 non-null    float64
 8   poster_path        200 non-null    object 
 9   release_date       200 non-null    object 
 10  title              200 non-null    object 
 11  video              200 non-null    bool   
 12  vote_average       200 non-null    float64
 13  vote_count         200 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 19.3+ KB


In [4]:
# popular_movies.to_csv("../data-files/popular_movies.csv", header=True, index=True)

In [5]:
popular_movies=popular_movies.drop(['adult', 'backdrop_path', 'video','genre_ids'], axis=1)

In [6]:
popular_movies.head(2)

,id,original_language,original_title,overview,popularity,poster_path,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,12506.888,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,4025
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,4404.247,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.9,1763


In [7]:
# 칼럼이름을 DB 칼럼이름과 동일하게 변경

popular_movies=popular_movies.rename(columns={'id':'movie_id', 'original_title':'subtitle','poster_path':'posterpath'})
popular_movies.head(2)

,movie_id,original_language,subtitle,overview,popularity,posterpath,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,12506.888,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,4025
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,4404.247,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.9,1763


In [8]:
popular_movies.isna().sum()

movie_id             0
original_language    0
subtitle             0
overview             0
popularity           0
posterpath           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [9]:
popular_movies_ids=popular_movies[["movie_id"]]
popular_movies_ids

,movie_id
0,580489
1,512195
2,585245
3,566525
4,589761
...,...
195,743439
196,866408
197,852663
198,15092


In [11]:
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

# cursor.execute('DELETE FROM popular_movies')
error_list=[]
for movie_id in popular_movies_ids.values:
    # print(type(movie_id), movie_id)  # movie_id type이 nd.arrary임.
    try:
        sql= "INSERT INTO popular_movies VALUES(%s)"
        cursor.execute(sql, movie_id[0]) 
    except:
        error_list.append(movie_id)
    
conn.commit()
cursor.close()
conn.close()

In [13]:
len(error_list)

17